In [1]:
import os
import sys
import json
import pandas as pd
import tensorflow as tf

from glob import glob
from tqdm import tqdm

In [2]:
def open_file(path):
    if 'csv' in path:
        print("csv file")
        result = pd.read_csv(path)
    elif 'json' in path:
        print("json file")
        with open(path, 'r') as f:
            result = json.load(f)
    else :
        return path
    return result

In [16]:
df = open_file('/home/jaeho/Documents/ai_pbl_final/data/val.csv')

csv file


In [17]:
df.head()

,image_path,bbox,labels
0,/media/jaeho/SSD/datasets/pet_data/val/cat/ima...,"192,646,527,971",편안/안정
1,/media/jaeho/SSD/datasets/pet_data/val/cat/ima...,"176,619,551,978",편안/안정
2,/media/jaeho/SSD/datasets/pet_data/val/cat/ima...,"210,622,463,950",편안/안정
3,/media/jaeho/SSD/datasets/pet_data/val/cat/ima...,"210,605,463,937",편안/안정
4,/media/jaeho/SSD/datasets/pet_data/val/cat/ima...,"206,586,493,975",편안/안정


In [18]:
err_row = {}
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_path = row['image_path']
    bbox_str = row['bbox']
    bbox = tf.strings.split(bbox_str, ",")
    bbox = tf.strings.to_number(bbox, tf.int32)
    img = tf.io.read_file(img_path)
    try:
        img = tf.io.decode_and_crop_jpeg(img, bbox)
    except:
        err_row[idx] = {
            'image_path' : img_path,
            'bbox' : bbox_str
        }
df = df.drop(err_row.keys(), axis=0)
df.to_csv('/home/jaeho/Documents/ai_pbl_final/data/val.csv', index=False)

100%|██████████| 757078/757078 [41:05<00:00, 307.13it/s]


In [9]:
len(err_row)

14

In [10]:
err_index = err_row.keys()
len(err_index)

14

In [11]:
len(df)

2127962

In [12]:
df = df.drop(index=err_index, axis=0)

In [13]:
len(df)

2127948

In [14]:
df.to_csv('/home/jaeho/Documents/ai_pbl_final/data/train.csv', index=False)

In [11]:
import pickle

In [12]:
with open('./image_err.pickle', 'wb') as f:
    pickle.dump(err_row, f, pickle.HIGHEST_PROTOCOL)